In [497]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [498]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

df = train.copy()
df['age'] = 2024 - df.model_year
test['age'] = 2024 - test.model_year

df.head(2)


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,age
id,,,,,,,,,,,,,
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000,6
1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250,17


In [481]:
test['age'] = 2024 - test.model_year

In [482]:
# df_X = df.iloc[:,:-1]
# df_y = df.iloc[:,-1]
df_y = df['price']
df_X = df.copy()
gg =df
# df_X.drop('price', axis=1, inplace = True)

In [483]:
df_X.drop('price', axis=1, inplace = True)


In [484]:
cols = list(df_X.columns)
objcols = [col for col in cols if df_X[col].dtype == 'object']
numcols = list(set(cols) - set(objcols))
print(f'All cols: {cols}')
print(f'Object cols: {objcols}')
print(f'Number cols: {numcols}')

All cols: ['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title', 'age']
Object cols: ['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']
Number cols: ['age', 'model_year', 'milage']


In [485]:
import re
def extract_features(sentence):
    hp = re.search(r'(\d+\.?\d*)HP', sentence)
    liters = re.search(r'(\d+\.?\d*)L', sentence)
    #cylinder = re.search(r'(\d+|\bV\d+\b) Cylinder', sentence)
    cylinders = re.search(r'(\d+\.?\d*)\s?(V\d|Straight\s?\d)?\s?Cylinder', sentence)
    
    hp_value = float(hp.group(1)) if hp else None
    liters_value = float(liters.group(1)) if liters else None
    #cylinder_value = cylinder.group(1) if cylinder else None
    cylinders_value = int(re.search(r'\d+', cylinders.group(0)).group(0)) if cylinders else None
    
    return pd.Series([hp_value, liters_value, cylinders_value])

In [486]:
# Calculate average price for each brand
average_price = df.groupby('brand')['price'].mean()
total_sum = df['price'].sum()
normalized_avg_price = average_price / total_sum
ranked_avg_price = normalized_avg_price.rank()

# Create a dictionary to map brands to their rank
brand_to_rank = ranked_avg_price.to_dict()

def get_brand_rank(brand, default_rank):
    return brand_to_rank.get(brand, default_rank)

# Calculate the default rank for unknown brands
default_rank = 0

In [487]:
def preprocessing(df_X):
    # Encode the brand column in the training DataFrame
    df_X['brand_encoded'] = df_X['brand'].apply(lambda x: get_brand_rank(x, default_rank))

    df_X.drop(columns =['clean_title', 'brand'], inplace = True)
    # Custom Encoding
    df_X[['HP', 'L', 'Cylinder_Count']] = df_X['engine'].apply(extract_features)
    #df_X['Cylinder_Count'].replace({'V6':7}, inplace = True)
    df_X.fillna({'HP': df_X['HP'].mode()[0]}, inplace=True)
    df_X.fillna({'L': df_X['L'].mode()[0]}, inplace=True)
    df_X.fillna({'Cylinder_Count': df_X['Cylinder_Count'].mode()[0]}, inplace=True)
    #df_X = fill_na_with_mode(df_X)
    #df_X['Cylinder_Count'] = df_X['Cylinder_Count'].astype(int)
    df_X.drop(columns =['engine'], inplace = True)
    
    # OneHot Encoding
    from sklearn.preprocessing import OneHotEncoder
    ohe_cols = ['fuel_type', 'accident']
    ohe = OneHotEncoder(drop='first', sparse_output = False)
    df_ohe = pd.DataFrame(ohe.fit_transform(df_X[ohe_cols]))
    df_ohe.index = df_X.index
    names = list(ohe.get_feature_names_out())
    df_X[names] = df_ohe
    df_X.drop(columns = ohe_cols, axis = 1, inplace = True)
    
    # Ordinal Encoding
    from sklearn.preprocessing import OrdinalEncoder
    ord_cols = ['model', 'transmission', 'ext_col', 'int_col']
    ordinal = OrdinalEncoder()
    df_X[ord_cols] = ordinal.fit_transform(df_X[ord_cols])
    
    df_X.dropna(inplace = True)
    
    #Standard Scaling
    from sklearn.preprocessing import StandardScaler
    cols = list(df_X.columns)
    scaler = StandardScaler()
    df_X[cols] = scaler.fit_transform(df_X[cols])
    
    
    '''from sklearn.decomposition import PCA
    pca = PCA(n_components=5)
    df_train = pca.fit_transform(df_train)'''
    
    return df_X

In [488]:
X = preprocessing(df_X)
X.head(3)

,model,model_year,milage,transmission,ext_col,int_col,age,brand_encoded,HP,L,Cylinder_Count,fuel_type_E85 Flex Fuel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_Plug-In Hybrid,fuel_type_not supported,fuel_type_–,accident_None reported
id,,,,,,,,,,,,,,,,,,
0,-0.454731,0.520325,0.031759,-2.181534,-1.006217,1.087814,-0.520325,0.119628,0.438013,-0.156985,-0.175715,-0.167375,0.312693,-0.183395,-0.058006,-0.008585,-0.073801,0.600302
1,-1.552786,-1.447877,0.143728,-0.768039,-1.098609,-0.489270,1.447877,0.350809,-0.281280,-0.532688,-0.175715,-0.167375,0.312693,-0.183395,-0.058006,-0.008585,-0.073801,0.600302
2,1.625114,-1.090022,0.371412,-1.121413,0.584990,-0.587838,1.090022,-0.227142,-0.281280,0.368998,1.172686,-0.167375,0.312693,-0.183395,-0.058006,-0.008585,-0.073801,0.600302


In [489]:
# X.drop('model_year', axis = 1, inplace=True)
X.drop(['ext_col', 'int_col', 'model_year', 'model', 'accident_None reported'], axis = 1, inplace=True)


In [490]:
X = X.iloc[:, 0:7]
X.drop('transmission', axis=1, inplace=True)

In [491]:
X.head()

,milage,age,brand_encoded,HP,L,Cylinder_Count
id,,,,,,
0,0.031759,-0.520325,0.119628,0.438013,-0.156985,-0.175715
1,0.143728,1.447877,0.350809,-0.281280,-0.532688,-0.175715
2,0.371412,1.090022,-0.227142,-0.281280,0.368998,1.172686
3,-1.393115,-1.236035,0.350809,0.054390,-0.532688,-0.175715
4,0.757966,2.521442,-2.423355,-1.240337,0.068436,-0.175715


In [492]:
y = df_y.copy()
y.head(2)

id
0    11000
1     8250
Name: price, dtype: int64

In [493]:
from sklearn.model_selection import train_test_split,KFold
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

In [494]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [495]:
def build_model():
    model = Sequential()
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mae'])
    return model

In [496]:
# K-Fold Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
val_losses = []

for train_index, val_index in kfold.split(X):
    X_train_fold, X_val_fold = X.loc[train_index], X.loc[val_index]
    y_train_fold, y_val_fold = y.loc[train_index], y.loc[val_index]
    
    model = build_model()
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.001)
    
    history = model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=50, 
                        validation_data=(X_val_fold, y_val_fold), 
                        callbacks=[early_stopping, reduce_lr])
    
    val_loss = model.evaluate(X_val_fold, y_val_fold)
    val_losses.append(val_loss)

print(f'Cross-Validation Losses: {val_losses}')
print(f'Mean Validation Loss: {np.sqrt(np.mean(val_losses))}')

Epoch 1/100
869/869 ━━━━━━━━━━━━━━━━━━━━ 4s 805us/step - loss: 7673204224.0000 - mae: 39232.8867 - val_loss: 5395296768.0000 - val_mae: 36217.6484 - learning_rate: 0.0100
Epoch 2/100
869/869 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 7095003136.0000 - mae: 35594.5820 - val_loss: 4766686208.0000 - val_mae: 29902.4102 - learning_rate: 0.0100
Epoch 3/100
869/869 ━━━━━━━━━━━━━━━━━━━━ 1s 673us/step - loss: 6668135424.0000 - mae: 28835.3887 - val_loss: 4216712960.0000 - val_mae: 23547.5547 - learning_rate: 0.0100
Epoch 4/100
869/869 ━━━━━━━━━━━━━━━━━━━━ 1s 609us/step - loss: 5199553024.0000 - mae: 21914.6152 - val_loss: 3807991040.0000 - val_mae: 17738.9023 - learning_rate: 0.0100
Epoch 5/100
869/869 ━━━━━━━━━━━━━━━━━━━━ 1s 612us/step - loss: 6486163456.0000 - mae: 18785.9492 - val_loss: 3665472512.0000 - val_mae: 15235.8867 - learning_rate: 0.0100
Epoch 6/100
869/869 ━━━━━━━━━━━━━━━━━━━━ 1s 626us/step - loss: 4788578816.0000 - mae: 17322.9375 - val_loss: 3570637568.0000 - val_mae: 16013.9072 

In [439]:
X.shape
#X.columns

(54273, 17)

In [440]:
# df_pred

In [416]:
df_pred = preprocessing(test)
predict = model.predict(df_pred)
output = pd.DataFrame(data = predict, columns = ['price'], index = test.index)
output.reset_index(inplace = True)
output.head(5)

1131/1131 ━━━━━━━━━━━━━━━━━━━━ 0s 317us/step


,id,price
0,54273,26743.457031
1,54274,23342.218750
2,54275,30394.468750
3,54276,62488.777344
4,54277,35007.628906


In [417]:
# output.to_csv('PredictionSat3.csv', index = False)

In [418]:
output.shape

(36183, 2)

In [419]:
output

,id,price
0,54273,26743.457031
1,54274,23342.218750
2,54275,30394.468750
3,54276,62488.777344
4,54277,35007.628906
...,...,...
36178,90451,69098.640625
36179,90452,19669.425781
36180,90453,13253.567383
36181,90454,53285.269531


In [420]:
ss = pd.read_csv('kagg_res.csv')

In [421]:
ss

,id,price
0,54273,28597.828
1,54274,29206.607
2,54275,41852.240
3,54276,37362.020
4,54277,51931.230
...,...,...
36178,90451,42290.773
36179,90452,12691.439
36180,90453,14742.021
36181,90454,46280.996
